In [ ]:
# import pandas as pd
# import os

# df = pd.read_csv("/home/rupam/DataAlpha/data/raw/social_media/r_wallstreetbets_posts.csv")  # replace with actual path
# # Convert timestamp to readable datetime

# df["created_date"] = pd.to_datetime(df["created_utc"], unit='s')
# # Filter only 2020 posts

# df_2020 = df[df["created_date"].dt.year == 2020]

# # Optional: Save to a new CSV for later use
# df_2020.to_csv("/home/rupam/DataAlpha/data/raw/social_media/df_2020.csv", index=False)

# print(df_2020[["created_date", "title"]].head())
# # Drop posts with missing titles
# df_2020 = df_2020[df_2020["title"].notna()]
# # Optional: Remove duplicates
# df_2020 = df_2020.drop_duplicates(subset="title")
# # Check how many posts
# print(f"Number of posts from 2020: {len(df_2020)}")
# print(df_2020.shape)
# print(df_2020.columns)
# print(df_2020.head(3))


/tmp/ipykernel_25887/206365085.py:4: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/rupam/DataAlpha/data/raw/social_media/r_wallstreetbets_posts.csv")  # replace with actual path


              created_date                                              title
579370 2020-12-31 23:57:53                         Selling Leaps - Vega Plays
579371 2020-12-31 23:57:47     “2021 is gonna be different.I can feel it.”🚀🚀🚀
579372 2020-12-31 23:57:47  Don't work for your money, make your money wor...
579373 2020-12-31 23:57:13             Those GE 100$ calls didn’t work out...
579374 2020-12-31 23:56:40                                 Some advice needed
Number of posts from 2020: 264758
(264758, 13)
Index(['id', 'title', 'score', 'author', 'author_flair_text', 'removed_by',
       'total_awards_received', 'awarders', 'created_utc', 'full_link',
       'num_comments', 'over_18', 'created_date'],
      dtype='object')
            id                                              title  score  \
579370  ko0zd1                         Selling Leaps - Vega Plays      1   
579371  ko0zb8     “2021 is gonna be different.I can feel it.”🚀🚀🚀      1   
579372  ko0zau  Don't work for your 

In [ ]:
# import pandas as pd

# # Scrape S&P 500 tickers from Wikipedia
# url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
# table = pd.read_html(url)[0]  # First table on the page

# # Get tickers from the Symbol column
# sp500_tickers = set(table["Symbol"].tolist())

# import re
# from collections import Counter

# tickers = []

# for title in df_2020["title"].dropna():
#     words = re.findall(r"\b[A-Z]{1,5}\b", title)
#     tickers.extend(words)

# # Count frequency
# all_counts = Counter(tickers)

# # Filter only those in known tickers
# cleaned_counts = {ticker: count for ticker, count in all_counts.items() if ticker in sp500_tickers}

# # Display top 20
# print(sorted(cleaned_counts.items(), key=lambda x: -x[1])[:500])


# print(len(cleaned_counts))


[('TSLA', 6542), ('DD', 4777), ('PLTR', 4111), ('A', 3945), ('MSFT', 1581), ('AMD', 1534), ('AAPL', 1236), ('ON', 781), ('AMZN', 778), ('F', 731), ('ALL', 581), ('DIS', 566), ('T', 565), ('BA', 493), ('IT', 444), ('J', 438), ('C', 418), ('NVDA', 411), ('NOW', 404), ('CCL', 366), ('V', 349), ('PSA', 329), ('MRNA', 297), ('ARE', 294), ('GILD', 271), ('D', 265), ('WMT', 260), ('MGM', 247), ('NFLX', 247), ('GE', 244), ('PFE', 230), ('DOW', 228), ('UPS', 202), ('GM', 197), ('INTC', 195), ('L', 187), ('UBER', 183), ('O', 173), ('DAY', 165), ('PCG', 160), ('RCL', 157), ('CRM', 153), ('DAL', 149), ('SBUX', 135), ('BAC', 132), ('NCLH', 128), ('K', 121), ('JPM', 118), ('UAL', 116), ('PM', 109), ('CVS', 108), ('COST', 107), ('LULU', 105), ('HAS', 103), ('MU', 102), ('HD', 97), ('TGT', 95), ('FDX', 94), ('SO', 91), ('LOW', 88), ('CSCO', 87), ('ABNB', 84), ('GOOG', 81), ('KO', 80), ('ENPH', 78), ('WYNN', 77), ('MCD', 70), ('PG', 69), ('CZR', 69), ('ES', 68), ('XOM', 68), ('CRWD', 67), ('JNJ', 64), 

In [ ]:
# import re

# # Function to extract S&P 500 tickers from title
# def extract_valid_tickers(title):
#     words = re.findall(r"\b[A-Z]{1,5}\b", str(title))
#     return [word for word in words if word in sp500_tickers]

# df_2020["tickers"] = df_2020["title"].apply(extract_valid_tickers)

# # Keep only posts that mention at least one valid ticker
# df_2020 = df_2020[df_2020["tickers"].map(len) > 0]


In [ ]:
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# analyzer = SentimentIntensityAnalyzer()
# df_2020["sentiment"] = df_2020["title"].apply(lambda x: analyzer.polarity_scores(str(x))["compound"])
# df_2020.shape

(37214, 15)

In [ ]:
# # Expand to one row per (ticker, post)
# rows = []

# for _, row in df_2020.iterrows():
#     for ticker in row["tickers"]:
#         rows.append({
#             "date": row["date"].date(),  # only date part
#             "ticker": ticker,
#             "sentiment": row["sentiment"]
#         })

# df_exploded = pd.DataFrame(rows)


In [46]:
import pandas as pd
import os
import re
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# -----------------------
# 1. Load Reddit Dataset
# -----------------------
# Update the path below as needed.
DATA_PATH = "/home/rupam/DataAlpha/data/raw/social_media/r_wallstreetbets_posts.csv"
df = pd.read_csv(DATA_PATH)

# -------------------------------
# 2. Convert UNIX Timestamp to Date
# -------------------------------
# Assume the column "created_utc" stores UNIX timestamps.
df["created_date"] = pd.to_datetime(df["created_utc"], unit='s')

# -------------------------------------
# 3. Filter Dataset for the Year 2020
# -------------------------------------
df_2020 = df[df["created_date"].dt.year == 2020].copy()

# (Optional) Save the raw 2020 posts for future use
OUTPUT_2020_PATH = "/home/rupam/DataAlpha/data/raw/social_media/df_2020.csv"
df_2020.to_csv(OUTPUT_2020_PATH, index=False)

# Preview a few rows
print("Preview of 2020 posts:")
print(df_2020[["created_date", "title"]].head())
print(f"Number of posts from 2020: {len(df_2020)}")
print("Columns:", df_2020.columns.tolist())

# ----------------------------------------------------
# 4. Clean Data: Remove Posts with Missing Titles & Duplicates
# ----------------------------------------------------
df_2020 = df_2020[df_2020["title"].notna()].copy()
df_2020 = df_2020.drop_duplicates(subset="title")

# -------------------------------------------
# 5. Load S&P 500 Tickers from Wikipedia
# -------------------------------------------
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
# Read the first table from Wikipedia containing S&P 500 companies
sp500_table = pd.read_html(url)[0]
# Extract tickers from the "Symbol" column and convert to a set
sp500_tickers = set(sp500_table["Symbol"].tolist())

print(f"Total S&P 500 tickers loaded: {len(sp500_tickers)}")

# ----------------------------------------------------
# 6. Define a Function to Extract Valid Tickers from Title
# ----------------------------------------------------
def extract_valid_tickers(title):
    # Find all words that are 1-5 uppercase letters
    words = re.findall(r"\b[A-Z]{1,5}\b", str(title))
    # Filter words based on membership in the S&P 500 tickers set
    return [word for word in words if word in sp500_tickers]

# Use .loc to safely assign new column "tickers" to avoid SettingWithCopyWarning.
df_2020.loc[:, "tickers"] = df_2020["title"].apply(extract_valid_tickers)

# --------------------------------------------------
# 7. Filter Only Posts That Mention at Least One Ticker
# --------------------------------------------------
df_2020 = df_2020[df_2020["tickers"].map(len) > 0].copy()

# (Optional) Display frequency counts to see which tickers are most common:
all_extracted = []
for tlist in df_2020["tickers"]:
    all_extracted.extend(tlist)
frequency_counts = Counter(all_extracted)
print("Top 20 extracted tickers:", frequency_counts.most_common(20))
print(f"Unique valid tickers found: {len(frequency_counts)}")

# -------------------------------------
# 8. Compute Sentiment Using VADER on Titles
# -------------------------------------
analyzer = SentimentIntensityAnalyzer()
df_2020.loc[:, "sentiment"] = df_2020["title"].apply(
    lambda x: analyzer.polarity_scores(str(x))["compound"]
)

# ---------------------------------------------
# 9. (Optional) Expand Posts to One Row per Ticker
# ---------------------------------------------
# Many posts mention multiple tickers. We expand the data so each row corresponds to one ticker.
expanded_rows = []
for _, row in df_2020.iterrows():
    for ticker in row["tickers"]:
        expanded_rows.append({
            "date": row["created_date"].date(),  # Save just the date part
            "ticker": ticker,
            "sentiment": row["sentiment"],
            "title": row["title"]  # optional: keep the original title for reference
        })

df_expanded = pd.DataFrame(expanded_rows)

# ---------------------------------------------
# 10. Aggregate Sentiment by Date and Ticker
# ---------------------------------------------
# For each date and ticker, compute the average sentiment.
df_sentiment_daily = df_expanded.groupby(["date", "ticker"]).agg({
    "sentiment": "mean"
}).reset_index()

print("Aggregated daily sentiment sample:")
print(df_sentiment_daily.head())

# -------------------------------------------------------------
# 11. Save the Final Processed Data for Later Model Training
# -------------------------------------------------------------
OUTPUT_PROCESSED_PATH = "/home/rupam/DataAlpha/data/raw/processed/reddit_sentiment_2020.csv"
df_sentiment_daily.to_csv(OUTPUT_PROCESSED_PATH, index=False)

print(f"Final processed data saved to: {OUTPUT_PROCESSED_PATH}")


/tmp/ipykernel_25887/1899384348.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH)


Preview of 2020 posts:
              created_date                                              title
579370 2020-12-31 23:57:53                         Selling Leaps - Vega Plays
579371 2020-12-31 23:57:47     “2021 is gonna be different.I can feel it.”🚀🚀🚀
579372 2020-12-31 23:57:47  Don't work for your money, make your money wor...
579373 2020-12-31 23:57:13             Those GE 100$ calls didn’t work out...
579374 2020-12-31 23:56:40                                 Some advice needed
Number of posts from 2020: 286130
Columns: ['id', 'title', 'score', 'author', 'author_flair_text', 'removed_by', 'total_awards_received', 'awarders', 'created_utc', 'full_link', 'num_comments', 'over_18', 'created_date']
Total S&P 500 tickers loaded: 503
Top 20 extracted tickers: [('TSLA', 6542), ('DD', 4777), ('PLTR', 4111), ('A', 3945), ('MSFT', 1581), ('AMD', 1534), ('AAPL', 1236), ('ON', 781), ('AMZN', 778), ('F', 731), ('ALL', 581), ('DIS', 566), ('T', 565), ('BA', 493), ('IT', 444), ('J', 438), ('C